In [0]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import pandas as pd

In [0]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [0]:
def get_next_url(soup):
    for i in soup.select('a'):
        i = str(i)
        if 'siguiente' in i:
            url = i
    for i in url.split():
        if 'href' in i:
            i = i.replace('\n','')
            i = i.replace('amp;','')
            i = i.replace('"','')
            i = i.replace('href=','')
            i = "http://clasificados.lavoz.com.ar"+i
            url = i          
    return url

In [0]:
def get_urls(soup):  
    urls = []
    for i in soup.select('a'):
        i = str(i)
        if 'http://clasificados.lavoz.com.ar/avisos/' in i:
            urls.append(i)
    for x, url in enumerate(urls):
        for i in url.split():
            if 'href' in i:
                i = i.replace('\n','')
                i = i.replace('amp;','')
                i = i.replace('"','')
                i = i.replace('href=','')
                url = i
        urls[x] = url
    return list(set(urls))

In [0]:
def get_row(url):
    html = simple_get(url)
    soup = BeautifulSoup(html)
    text = soup.get_text().splitlines()
    value = ''
    x = {}
    
    for i in text:
        if 'U$S'in i:
            value = i
            value = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", value)
            value = ''.join(value)
            value = value.replace('.', '')
        elif '$'in i:
            value = i
            value = re.findall("[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", value)
            value = ''.join(value)
            value = value.replace('.', '')
            if value != '':
              value = str(int(int(value)/38.800))
        
        categories = ['Tipo de Propiedad:',
                      'Zona:',
                      'Ciudad:', 
                      'Barrio:', 
                      'Calle:', 
                      'Tipo de Barrio:',
                      'Apto Escritura:', 
                      'Apto Crédito:',
                      'Etapa de Construcción:', 
                      'Antigüedad:', 
                      'Cantidad de Plantas:',
                      'Cantidad de Dormitorios:',
                      'Cantidad de Baños:',
                      'Cantidad de Cocheras:',
                      'Cobertura Cocheras:',
                      'Superficie Total m2:',
                      'Superficie Cubierta m2:'
                     ]

        for j in categories:
            if j in i:
                if i.find(':') == j.find(':'):
                    i = re.sub('^(.*)(?=:)', "", i)
                    i = i.replace(': ','')
                    x[j.replace(':','')] = i 

    if value != '':
        x['Valor'] = value

    return x

In [0]:
def get_data(url, search):
  row = [] 
  try:
    for x in range(search):
      try:
        soup = BeautifulSoup(simple_get(url))
        urls = get_urls(soup)
      except:
        print('1: ',url)

      for i in urls:
        try:
          row.append(get_row(i))
        except:
          print('2: ', i)
      try:
        url = get_next_url(soup)
      except:
        print('3: ',url)
      print(x)   
    return row
  except:
    return row

In [0]:
url = 'http://clasificados.lavoz.com.ar/search/apachesolr_search?f[0]=im_taxonomy_vid_34%3A6330&f[1]=ss_operacion%3AVenta&f[2]=im_taxonomy_vid_34%3A6331'
x = get_data(url, 900)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
2:  http://clasificados.lavoz.com.ar/avisos/casas/3200279/va-cabrera-3-dor-garaje-patio-chico.html
156
157
158
159
160
161
162
163
164
165
2:  http://clasificados.lavoz.com.ar/avisos/casas/3070030/reciclar-increible-ubicacion-mejor-inversion-es-aca.html
2:  http://clasificados.lavoz.com.ar/avisos/casas/3172465/casa-dos-plantas-nuevas-quintas-sur.html
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
1

In [0]:
columns = ['Tipo de Propiedad',
           'Zona',
           'Ciudad', 
           'Barrio', 
           'Calle', 
           'Tipo de Barrio',
           'Apto Escritura', 
           'Apto Crédito:',
           'Etapa de Construcción', 
           'Antigüedad', 
           'Cantidad de Plantas',
           'Cantidad de Dormitorios',
           'Cantidad de Baños',
           'Cantidad de Cocheras',
           'Cobertura Cocheras',
           'Superficie Total m2',
           'Superficie Cubierta m2',
           'Valor'
          ]

dataFrame = pd.DataFrame(x, columns=columns)
dataFrame.head()

,Tipo de Propiedad,Zona,Ciudad,Barrio,Calle,Tipo de Barrio,Apto Escritura,Apto Crédito:,Etapa de Construcción,Antigüedad,Cantidad de Plantas,Cantidad de Dormitorios,Cantidad de Baños,Cantidad de Cocheras,Cobertura Cocheras,Superficie Total m2,Superficie Cubierta m2,Valor
0,Duplex,NaN,Córdoba,Claros del Bosque,Claros Del Bosque 5000,NaN,NaN,NaN,NaN,NaN,2 Plantas,3 Dormitorios,3 Baños,1,Cubierta,180,160,185000
1,Duplex,Sur,Córdoba,Greenville 2,NaN,Con Seguridad,NaN,NaN,NaN,NaN,NaN,3 Dormitorios,NaN,NaN,NaN,NaN,NaN,225000
2,Casa,NaN,Villa Allende,La Herradura,01 1,Country,NaN,NaN,NaN,NaN,2 Plantas,3 Dormitorios,3 Baños,2,Cubierta,1592,400,380000
3,Casa,Norte,Córdoba,Villa Belgrano,NaN,Abierto,Si,NaN,NaN,25 años,1 Planta,3 Dormitorios,4 Baños,2,Cubierta,1.200 mts2,446 mts2,420000
4,Casa,NaN,Córdoba,Maipu Sección 2,Benjamin Igarzabal 1300,NaN,NaN,NaN,NaN,NaN,NaN,4 Dormitorios,4 Baños,2,Cubierta,350,316,250000
5,Casa,NaN,Villa Allende,San Isidro,Consultar dirección,NaN,NaN,NaN,NaN,NaN,NaN,3 Dormitorios,3 Baños,NaN,NaN,1,1,620000
6,Casa,NaN,Villa Carlos Paz,NaN,Ayacucho esquina Colombia,NaN,Si,NaN,NaN,NaN,1 Planta,2 Dormitorios,2 Baños,3,NaN,900,150,200000
7,Duplex,Sur,Córdoba,Claros del Bosque,NaN,Con Seguridad,Si,NaN,En Construcción,NaN,1 Planta,2 Dormitorios,1 Baño,1,Cubierta,NaN,125,NaN
8,Casa,NaN,Córdoba,Maipu Sección 2,Malaga 1900,NaN,NaN,NaN,NaN,NaN,2 Plantas,3 Dormitorios,2 Baños,1,Cubierta,296,221,135000
9,Casa,NaN,Córdoba,Siete Soles,NaN,Cerrado,Si,NaN,NaN,NaN,1 Planta,3 Dormitorios,NaN,NaN,NaN,NaN,213,NaN


In [0]:
dataFrame.to_csv('Casas_venta.csv', encoding='utf_8_sig')